In [1]:
from positions import *
from ultralytics import YOLO
import os
import time

pieces_model_weights = 'runs/pieces/yolo11m_k2_pt/weights/best.pt'
board_model_weights = 'runs/board/yolo11n-seg_k0_pt/weights/best.pt'

TEST_FOLDER_IMAGES = 'dataset/test/images'
TEST_FOLDER_LABELS = 'dataset/test/correct_output'

pieces_model = YOLO(pieces_model_weights)
board_model = YOLO(board_model_weights)

images = [os.path.join(TEST_FOLDER_IMAGES, f) for f in os.listdir(TEST_FOLDER_IMAGES) if f.endswith('.png')]

classes = {
    'B': 0, 'K': 1, 'N': 2, 'P': 3, 'Q': 4, 'R': 5,
    'b': 6, 'k': 7, 'n': 8, 'p': 9, 'q': 10, 'r': 11
}
inv_classes = {v: k for k, v in classes.items()}

In [2]:
# Given a list of pieces in the format
# [{'piece': 'b', 'square': 'g1}, ...],
def pieces_to_fen(pieces):
    fen = ''
    board = [['.' for _ in range(8)] for _ in range(8)]
    
    for piece in pieces:
        square = piece['square']
        row = 8 - int(square[1])
        col = ord(square[0]) - ord('a')
        board[row][col] = piece['piece']
    
    for row in board:
        empty_count = 0
        for cell in row:
            if cell == '.':
                empty_count += 1
            else:
                if empty_count > 0:
                    fen += str(empty_count)
                    empty_count = 0
                fen += cell
        if empty_count > 0:
            fen += str(empty_count)
        fen += '/'
    
    return fen[:-1]  # Remove the last '/'

# Check if a piece is in a list of pieces
def piece_in_list(piece, pieces):
    for p in pieces:
        if p['piece'] == piece['piece'] and p['square'] == piece['square']:
            return True
    return False
    

In [4]:
import cv2
import numpy as np
import json

# [Correct, Wrong]
stats = {
    'fen': [0, 0],
    'pieces_pred': [0, 0],
    'pieces_real': [0, 0]
}

# Timing statistics
timing_stats = {
    'pieces_inference': [],
    'board_inference': [],
    'total_processing': []
}

for class_name in classes:
    stats[class_name + '_pred'] = [0, 0]
    stats[class_name + '_real'] = [0, 0]

many_errors_pred_board = []
many_errors_real_board = []
wrong_board = []
for i, image in enumerate(images):
    print(f'Processing {i+1}/{len(images)}: {image}')
    print(f'{stats["fen"]}, {stats["pieces_pred"]}, {stats["pieces_real"]}')
    
    start_total = time.time()

    # Read the label file
    label_file = os.path.join(TEST_FOLDER_LABELS, os.path.basename(image).replace('.png', '.json'))
    if not os.path.exists(label_file):
        print(f'Label file not found: {label_file}')
        continue

    with open(label_file, 'r') as f:
        label_data = json.load(f)
        
    # Detect pieces with timing
    start_pieces = time.time()
    pieces_results = pieces_model.predict(image, conf=0.5, verbose=False, save=False)
    pieces_time = time.time() - start_pieces
    timing_stats['pieces_inference'].append(pieces_time)
    #print(f'Pieces inference time: {pieces_time:.4f}s')
    
    # Detect board with timing
    start_board = time.time()
    board_results = board_model.predict(image, conf=0.5, verbose=False, save=False)
    board_time = time.time() - start_board
    timing_stats['board_inference'].append(board_time)
    #print(f'Board inference time: {board_time:.4f}s')

    detected_pieces = []
    correct_pieces = label_data['pieces']

    # Process results
    if board_results[0].masks.xy is not None and len(board_results[0].masks.xy) > 0:
        # Get the original mask contours from xy coordinates
        mask_contours = board_results[0].masks.xy[0]
        
        # Convert to numpy array for OpenCV operations
        contour_points = np.array(mask_contours, dtype=np.float32)
        
        # Approximate the contour to a quadrilateral using masks.xy
        epsilon = 0.05 * cv2.arcLength(contour_points, True)
        board_vert = cv2.approxPolyDP(contour_points, epsilon, True)
        while len(board_vert) != 4:
            if len(board_vert) > 4:
                epsilon *= 1.05
            else:
                epsilon *= 0.95
            board_vert = cv2.approxPolyDP(contour_points, epsilon, True)
            rep += 1

            if rep > 100:
                print('Too many iterations, breaking out')
                break
        if len(board_vert) == 4:
            board_vert = board_vert.reshape(-1, 2)  # Flatten to 2D array
            board_vert = [[float(x), float(y)] for x, y in board_vert]


            transform = calc_transform(board_vert)
        
            # Process pieces results
            if len(pieces_results) > 0:
                for j in range(len(pieces_results[0].boxes.cls)):
                    box = [float(pieces_results[0].boxes.xyxy[j][0]), float(pieces_results[0].boxes.xyxy[j][1]),
                        float(pieces_results[0].boxes.xywh[j][2]), float(pieces_results[0].boxes.xywh[j][3])]
                    
                    piece = {
                        'piece' : inv_classes[int(pieces_results[0].boxes.cls[j])],
                        'box' : box,
                        'square' : calc_position(box, transform, 0 if label_data["white_turn"] else 2)
                    }
                    
                    if not 'ERROR' in piece['square']:
                        detected_pieces.append(piece)
        
        wrong_pieces_pred = 0
        for piece in detected_pieces:
            if piece_in_list(piece, correct_pieces):
                stats['pieces_pred'][0] += 1
                stats[piece['piece'] + '_pred'][0] += 1
            else:
                wrong_pieces_pred += 1
                stats['pieces_pred'][1] += 1
                stats[piece['piece'] + '_pred'][1] += 1
        
        wrong_pieces_real = 0
        for piece in correct_pieces:
            if piece_in_list(piece, detected_pieces):
                stats['pieces_real'][0] += 1
                stats[piece['piece'] + '_real'][0] += 1
            else:
                wrong_pieces_real += 1
                stats['pieces_real'][1] += 1
                stats[piece['piece'] + '_real'][1] += 1
        
        # Convert detected pieces to FEN
        fen_pred = pieces_to_fen(detected_pieces)
        fen_real = label_data['fen']

        if fen_pred == fen_real:
            stats['fen'][0] += 1
        else:
            wrong_board.append(image)
            stats['fen'][1] += 1

        if wrong_pieces_pred > 5:
            many_errors_pred_board.append(image)
        if wrong_pieces_real > 5:
            many_errors_real_board.append(image)

    total_time = time.time() - start_total
    timing_stats['total_processing'].append(total_time)
    #print(f'Total processing time: {total_time:.4f}s\n')
    

# print wrong boards for debugging
print(f"Boards with many errors in predicted pieces ({len(many_errors_pred_board)}):", many_errors_pred_board)
print(f"Boards with many errors in real pieces ({len(many_errors_real_board)}):", many_errors_real_board)
print(f"Boards with wrong FEN ({len(wrong_board)}):", wrong_board)

print("Statistics:")
for key, value in stats.items():
    print(f"{key}: Correct: {value[0]}, Wrong: {value[1]} ({(value[0] / (value[0] + value[1])) * 100:.2f}%)")
print(f"Total pieces detected: {stats['pieces_pred'][0] + stats['pieces_pred'][1]}")
print(f"Total pieces real: {stats['pieces_real'][0] + stats['pieces_real'][1]}")
print(f"Total FEN correct: {stats['fen'][0] + stats['fen'][1]}")
print(f"Total FEN correct: {stats['fen'][0]} / {stats['fen'][0] + stats['fen'][1]} ({(stats['fen'][0] / (stats['fen'][0] + stats['fen'][1])) * 100:.2f}%)")
print(f"Total pieces predicted: {stats['pieces_pred'][0] + stats['pieces_pred'][1]}")
print(f"Total pieces real: {stats['pieces_real'][0] + stats['pieces_real'][1]}")

print("\nTiming Statistics:")
if timing_stats['pieces_inference']:
    pieces_avg = sum(timing_stats['pieces_inference']) / len(timing_stats['pieces_inference'])
    print(f"Pieces model - Avg: {pieces_avg:.4f}s, Min: {min(timing_stats['pieces_inference']):.4f}s, Max: {max(timing_stats['pieces_inference']):.4f}s")

if timing_stats['board_inference']:
    board_avg = sum(timing_stats['board_inference']) / len(timing_stats['board_inference'])
    print(f"Board model - Avg: {board_avg:.4f}s, Min: {min(timing_stats['board_inference']):.4f}s, Max: {max(timing_stats['board_inference']):.4f}s")

if timing_stats['total_processing']:
    total_avg = sum(timing_stats['total_processing']) / len(timing_stats['total_processing'])
    print(f"Total processing - Avg: {total_avg:.4f}s, Min: {min(timing_stats['total_processing']):.4f}s, Max: {max(timing_stats['total_processing']):.4f}s")

Processing 1/1888: dataset/test/images/2357.png
[0, 0], [0, 0], [0, 0]
Processing 2/1888: dataset/test/images/3241.png
[1, 0], [30, 0], [30, 0]
Processing 3/1888: dataset/test/images/4636.png
[2, 0], [35, 0], [35, 0]
Processing 4/1888: dataset/test/images/4865.png
[3, 0], [62, 0], [62, 0]
Processing 5/1888: dataset/test/images/1691.png
[4, 0], [85, 0], [85, 0]
Processing 6/1888: dataset/test/images/4693.png
[4, 1], [99, 10], [99, 10]
Processing 7/1888: dataset/test/images/2923.png
[5, 1], [121, 10], [121, 10]
Processing 8/1888: dataset/test/images/0240.png
[6, 1], [149, 10], [149, 10]
Processing 9/1888: dataset/test/images/4199.png
[7, 1], [163, 10], [163, 10]
Processing 10/1888: dataset/test/images/2466.png
[8, 1], [190, 10], [190, 10]
Processing 11/1888: dataset/test/images/4654.png
[9, 1], [202, 10], [202, 10]
Processing 12/1888: dataset/test/images/0913.png
[10, 1], [218, 10], [218, 10]
Processing 13/1888: dataset/test/images/0742.png
[11, 1], [248, 10], [248, 10]
Processing 14/188

In [5]:
import json
stats_file = 'test_stats.json'
with open(stats_file, 'w') as f:
    json.dump(stats, f, indent=2)